# CLIMADA - simple_wind_denmark data flow: EXAMPLE 1

In this notebook we show one simple impact calculations with CLIMADA and its data API.

In [1]:
from climada.util.api_client import Client
client = Client()

/Users/ckropf/opt/anaconda3/envs/climada_env/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 12.0.1. Please consider upgrading.
  warnings.warn(


First, we load the hazard dataset (which contains flood depths for each day between 1971 and 2023 for a total of 19358 events) from the PIK.

In [2]:
haz = client.get_hazard(
    hazard_type='storm_europe',
    properties={
        'climate_scenario':'ssp585',
        'spatial_coverage':'Europe',
        'data_source':'CMIP6',
        'gcm':'CMCC-ESM2'
    }
)

Second, we load an exposure file downloaded from WorldPop

In [3]:
from climada.entity import Exposures
from pathlib import Path
exp_path = Path('../Data/')
exp_file = 'dnk_ppp_2020_1km_Aggregated_UNadj.tif' # Worldpop population distribution for Denmark
exp = Exposures.from_raster(
    exp_path / exp_file,
)

Third, we define an impact function. This is tricky, as this typically depends strongly on the datasets, and the user should define them carefully. Here we simply assume a 1m threshold for certain countries, and 2m for others (just for demonstration)

In [4]:
from climada.entity import ImpactFunc, ImpactFuncSet
min_wind = 0
max_wind = 80

threshold1 = 30
impf_id1 = 1
impf1 = ImpactFunc.from_step_impf(
    intensity=(min_wind, threshold1, max_wind),
    haz_type=haz.haz_type,
    impf_id=impf_id1
)

impfset = ImpactFuncSet([impf1])

Fourth, we need to assign the impact functions to the exposure points. Here we choose certain countries at random to have 1m, others 2m.

In [5]:
exp.gdf[f'impf_{haz.haz_type}'] = impf_id1

In [6]:
# We can assign centroids of the hazard the exposure points manually. This is useful when doing repeated impact calculations
exp.assign_centroids(haz, threshold=1.5) #threshold for nearest neighbour assignement is chosen to 1.5km as the population dataset is a 1km resolution (arbitrary choice, default would be 100)

2024-08-05 14:40:26,611 - climada.util.coordinates - WARNING - Distance to closest centroid is greater than 1.5km for 328170 coordinates.


Fith, we compute the impacts

In [7]:
from climada.engine import ImpactCalc
impcalc = ImpactCalc(exposures=exp, impfset=impfset, hazard=haz)

In [8]:
impact = impcalc.impact(assign_centroids=False)

Sixth, we can extract diverse risk and impact metrics. One can also save them to file.

In [9]:
# Average annual impact over the whole region
impact.aai_agg

3130.063295062383

In [10]:
# Average annual impact per exposure point
impact.eai_exp

array([0., 0., 0., ..., 0., 0., 0.])

In [11]:
# Total impact per event
impact.eai_exp

array([0., 0., 0., ..., 0., 0., 0.])

In [12]:
# Impact at given return periods
impact.calc_freq_curve(return_per=[1,2,5,10,20])

ImpactFreqCurve(return_per=[1, 2, 5, 10, 20], impact=array([ 288.0693593 , 3222.00770569, 5110.56558537, 5398.63494468,
       5398.63494468]), unit='USD', frequency_unit='1/year', label='Exceedance frequency curve')

In [13]:
# Save to csv (netcdf will be implemented soon)
impact.write_csv('impact_simple_wind_denmark_example1.csv')

In [14]:
# The full event/point impact matrix
impact.imp_mat

<3090x328320 sparse matrix of type '<class 'numpy.float64'>'
	with 1463 stored elements in Compressed Sparse Row format>

In [15]:
# Save the full event/point impact matrix in sparse format
impact.write_sparse_csr('impact_simple_wind_denmark_example1_impmat.npz')